In [28]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

In [29]:
namespace = utils.get_default_target_namespace()
print(namespace)

admin


In [30]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://kfserving-samples/models/tensorflow/flowers',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flower-sample', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

In [31]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2020-08-29T08:33:04Z',
  'generation': 1,
  'name': 'flower-sample',
  'namespace': 'admin',
  'resourceVersion': '56868',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/admin/inferenceservices/flower-sample',
  'uid': '40d0aa4e-2b5b-4d42-ab51-346e73fd50bd'},
 'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '100m',
       'memory': '1Gi'},
      'requests': {'cpu': '100m', 'memory': '1Gi'}},
     'runtimeVersion': '1.14.0',
     'storageUri': 'gs://kfserving-samples/models/tensorflow/flowers'}}}}}

In [32]:
KFServing.get('flower-sample', namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
flower-sample        False                                                                                        
flower-sample        False                                                                                        
flower-sample        True       100                             http://flower-sample.admin.example.com/v1/model...


In [7]:
canary_endpoint_spec = V1alpha2EndpointSpec(
                         predictor=V1alpha2PredictorSpec(
                           tensorflow=V1alpha2TensorflowSpec(
                             storage_uri='gs://kfserving-samples/models/tensorflow/flowers-2',
                             resources=V1ResourceRequirements(
                                 requests={'cpu':'100m','memory':'1Gi'},
                                 limits={'cpu':'100m', 'memory':'1Gi'}))))

KFServing.rollout_canary('flower-sample', canary=canary_endpoint_spec, percent=10,
                         namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
flower-sample        False                                                                                        
flower-sample        True       90              10              http://flower-sample.admin.example.com/v1/model...


In [8]:
KFServing.rollout_canary('flower-sample', percent=50, namespace=namespace,
                         watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC  CANARY_TRAFFIC URL                                               
flower-sample        True                    50              50 http://flower-sample.admin.example.com/v1/model...


In [9]:
KFServing.promote('flower-sample', namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC  CANARY_TRAFFIC URL                                               
flower-sample        True                    50              50 http://flower-sample.admin.example.com/v1/model...


In [33]:
KFServing.delete('flower-sample', namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'flower-sample',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': '40d0aa4e-2b5b-4d42-ab51-346e73fd50bd'}}